# Домашнее задание 3. 

## Предсказание пользовательской оценки отеля по тексту отзыва.

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели - секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача - научиться предсказывать оценку отеля по отзыву. Данные можно скачать [тут](https://www.kaggle.com/c/hseds-texts-2020/data?select=train.csv).

Главная метрика - Mean Absolute Error (MAE). Во всех частях домашней работы вам нужно получить значение MAE не превышающее 1. В противном случае мы будем вынуждены не засчитать задание :( 

Для измерения качества вашей модели используйте разбиение данных на train и test и замеряйте качество на тестовой части.

#### Про данные:
Каждое ревью состоит из двух текстов: positive и negative - плюсы и минусы отеля. В столбце score находится оценка пользователя - вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Удачи! 💪

#### Использовать внешние данные для обучения строго запрещено. Можно использовать предобученные модели из torchvision.

In [3]:
# imports

import os
import re
import sys
import nltk
import torch
import string
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from torch import cuda
from google.colab import files
from torch.nn import functional as F
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mariamanakhova","key":"bed2ca6fda8d9407a73f8e0104d14491"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [6]:
!pip install -q kaggle
!kaggle competitions download -c hseds-texts-2020
!unzip -q train.csv.zip -d .
!unzip -q test.csv.zip -d .

  0% 0.00/723k [00:00<?, ?B/s]
100% 723k/723k [00:00<00:00, 49.5MB/s]
  0% 0.00/1.75M [00:00<?, ?B/s]
100% 1.75M/1.75M [00:00<00:00, 118MB/s]
  0% 0.00/8.78M [00:00<?, ?B/s]
100% 8.78M/8.78M [00:00<00:00, 139MB/s]


In [7]:
PATH_TO_TRAIN_DATA = 'train.csv'

In [8]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7


Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [9]:
nltk.download('punkt')

def process_text(text):
    return re.sub(r'\d+', '', ' '.join([word for word in word_tokenize(text.lower()) if word not in string.punctuation]))  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
df['negative'] = df['negative'].apply(process_text)
df['positive'] = df['positive'].apply(process_text)

In [11]:
df_train, df_test = train_test_split(df, random_state = 42)

In [12]:
df_train.head()

,review_id,negative,positive,score
98980,fd5b30514ba1d0132c5c549fd877b131,cups washed up each day they stayed dirty all ...,no positive,4.2
69824,b2e77862f1831febc2bef16da8a9a1e3,tiny but tidy cosy pool,good location friendly staff,8.8
9928,19ee6dfdfaa1237792e8f1fe51b32796,no negative,everything good location really nice room,8.3
75599,c1b08b95b99b95c8587aadd6a126b67d,the walls between the rooms are a bit thin you...,the linen was always fresh never felt like any...,10.0
95621,f4babeb406b42b3cb0889ef6c7e7535a,nothing,excellent location beside the underground,9.6


### Часть 1. 1 балл

Обучите логистическую регрессию на TF-IDF векторах текстов.

Так как score – дробные числа, то проще обучать линейную регрессию.

In [13]:
def split_data(df_train, df_test):
    X_train = df_train['negative'] + ' ' + df_train['positive']
    X_test = df_test['negative'] + ' ' + df_test['positive']
    y_train = df_train['score']
    y_test = df_test['score']

    return X_train, y_train, X_test, y_test

In [14]:
X_train, y_train, X_test, y_test = split_data(df_train, df_test)

In [15]:
vectorizer = TfidfVectorizer(stop_words='english')

In [16]:
vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [17]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [18]:
def train_and_evaluate(X_train, y_train, X_test, y_test):
    regressor = SGDRegressor()
    regressor.fit(X_train, y_train)
    y_predicted = regressor.predict(X_test)

    return mean_absolute_error(y_test, y_predicted)

In [19]:
f'mae = {train_and_evaluate(X_train, y_train, X_test, y_test)}'

'mae = 0.988401612985892'

### Часть 2. 3 балла

Обучите логистическую регрессию на усредненных Word2Vec векторах. 

Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (Inverse document frequency)

In [20]:
def calc_idf(texts):
    pass

Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга. 
#### Сделайте выводы:

Теперь попробуйте обучить логистическую регрессию на любых других эмбеддингах размерности 300 и сравните качество с Word2Vec.
#### Выводы:

### Часть 3. 6 баллов

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки. Получите ошибку меньше, чем во всех вышеперечисленных методах.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинные относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример pad_sequence)

In [50]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)

In [51]:
WORDS = set()
for sent in list(df['positive']):
    for w in sent:
        WORDS.add(w)
        
for sent in list(df['negative']):
    for w in sent:
        WORDS.add(w)

In [52]:
int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: ii for ii, w in int2word.items()}

In [69]:
MAX_LENGTH = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))
MAX_LENGTH

1965

In [70]:
def stack(x, y, size):
    x.extend(y)
    return torch.as_tensor(x[:size])

In [71]:
def process_with_pad_sequence(df, embedding_length):
    X_positive = [[word2int[w] for w in seq][:embedding_length] for seq in df['positive']]
    X_negative = [[word2int[w] for w in seq][:embedding_length] for seq in df['negative']]

    X = [stack(X_positive[i], X_negative[i], embedding_length) for i in range(len(X_positive))]
    y = df['score']

    return torch.LongTensor(pad_sequence(X, batch_first=True)), torch.LongTensor(y.to_numpy())

In [72]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df, embedding_length = 300):
        self.reviews, self.scores = process_with_pad_sequence(df, embedding_length)
        
    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, idx):
        return self.reviews[idx], self.scores[idx]

In [73]:
def get_device():
    if cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [74]:
BATCH_SIZE = 32    
device = get_device()

df_train, df_test = train_test_split(df, random_state = 42)

train_dataset = ReviewsDataset(df_train)
test_dataset = ReviewsDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [62]:
class LSTM(torch.nn.Module):
    def __init__(self, n_targets, num_embeddings, embedding_dim, n_hidden):
        super().__init__()
        self.lstm = nn.LSTM(embedding_dim, n_hidden, batch_first = True)
        self.embeddings = nn.Embedding(num_embeddings, embedding_dim, padding_idx = 0)
        self.linear = nn.Linear(n_hidden, n_targets)
        
    def forward(self, x):
        out, _ = self.lstm(self.embeddings(x.to(device)))

        return self.linear(out.sum(1))

In [63]:
def mae(output_score, test_score):
    _, predicted_score = torch.max(output_score.data.detach().cpu(), dim = 1)

    return mean_absolute_error(predicted_score, test_score.detach().cpu())

In [64]:
def train(model, train_dataloader, test_dataloader, num_epochs = 30, lr = 0.001, clip = 1):
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    model.to(device)

    for epoch in range(num_epochs):
        print(f'epoch {epoch + 1}')
        with tqdm(total = len(train_dataloader), file = sys.stdout, position = 0) as prbar:
            for batch in train_dataloader:
                input = batch[0].to(device)
                score = batch[1].to(device)
          
                model.zero_grad()
                output = model(input)
                
                loss = criterion(output, score.long()) 
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()
                
                batch_loss = round(loss.detach().cpu().item(), 3)
                batch_mae = round(mae(output, score), 3)

                prbar.set_description(f'[TRAIN] mae = {batch_mae} | loss = {batch_loss}') 
                prbar.update(1)

        test_mae = 0

        with torch.no_grad():
            with tqdm(total = len(test_dataloader), file = sys.stdout, position = 0) as prbar:
                for batch in test_dataloader:
                    input = batch[0].to(device)
                    score = batch[1].to(device)
                  
                    output = model(input).detach().cpu()

                    batch_mae = round(mae(output, score), 3)

                    prbar.set_description(f'[TEST]  mae = {round(batch_mae, 3)}')
                    prbar.update(1)

                    test_mae += batch_mae

        mae_on_test = round(test_mae / len(test_dataloader), 3)
        print(f'mae on test = {mae_on_test}\n')

        if (mae_on_test < 1.0):
            break


Так как в выборке score – вещественное число от 0 до 10 с точностью один знак после запятой, тогда получим 10 / 0.1 + 1 = 101 таргетов.

In [65]:
model = LSTM(n_targets = 101, num_embeddings = len(int2word), embedding_dim = 300, n_hidden = 300)
train(model, train_dataloader, test_dataloader)

[TRAIN] mae = 1.0 | loss = 1.422: 100%|██████████| 2344/2344 [01:06<00:00, 35.11it/s]
[TEST]  mae = 1.125: 100%|██████████| 782/782 [00:08<00:00, 93.70it/s]
mae on test = 1.112
[TRAIN] mae = 0.958 | loss = 1.415: 100%|██████████| 2344/2344 [01:06<00:00, 35.27it/s]
[TEST]  mae = 1.125: 100%|██████████| 782/782 [00:08<00:00, 96.45it/s]
mae on test = 1.076
[TRAIN] mae = 1.0 | loss = 1.448: 100%|██████████| 2344/2344 [01:06<00:00, 35.24it/s]
[TEST]  mae = 0.625: 100%|██████████| 782/782 [00:08<00:00, 93.75it/s]
mae on test = 1.052
[TRAIN] mae = 0.833 | loss = 1.46: 100%|██████████| 2344/2344 [01:06<00:00, 35.20it/s]
[TEST]  mae = 0.625: 100%|██████████| 782/782 [00:08<00:00, 93.88it/s]
mae on test = 1.035
[TRAIN] mae = 0.875 | loss = 1.444: 100%|██████████| 2344/2344 [01:06<00:00, 34.99it/s]
[TEST]  mae = 0.625: 100%|██████████| 782/782 [00:08<00:00, 95.25it/s]
mae on test = 1.02
[TRAIN] mae = 0.917 | loss = 1.461: 100%|██████████| 2344/2344 [01:06<00:00, 35.29it/s]
[TEST]  mae = 0.75: 100

Если долго ~~мучаться~~ обучать, то рано или поздно mae становится где-то 0.99. В среднем необходимо от 17 до 25 эпох, чтобы добиться такого результата.



### Бонус. 10 баллов

Побейте качество 0.75 в [соревновании](https://www.kaggle.com/c/hseds-texts-2020/leaderboard). Можете воспользоваться вышеперечисленными методами или попробовать что-нибудь еще.